<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Chest%20X_Ray/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandas tensorflow
# !pip install seaborn

In [ ]:
# # Install latex
# !sudo apt-get update -y
# !sudo apt-get install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super

In [ ]:
# !pip install tf-slim

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Define your data directory path and image size
data_dir = "/content/drive/MyDrive/Chest X_Ray"
mobilenet_input_size = (224, 224)  # Image size for MobileNetV2

# Load and resize image
def load_image(directory, cls, img_file):
    img_path = os.path.join(directory, cls, img_file)
    img = Image.open(img_path).resize(mobilenet_input_size)
    return img

# Prepare data
X, y = [], []
label_map = {cls: idx for idx, cls in enumerate(os.listdir(data_dir)) if os.path.isdir(os.path.join(data_dir, cls))}

for cls in label_map.keys():
    cls_dir = os.path.join(data_dir, cls)
    for img_file in os.listdir(cls_dir):
        img_path = os.path.join(cls_dir, img_file)
        img = load_img(img_path, target_size=mobilenet_input_size)
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(label_map[cls])

X = np.array(X).astype('float32') / 255.0
y = np.array(y).astype('int')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=22)

# Define MobileNetV2 model
def build_mobilenetv2(input_shape, num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

# Build and compile model
input_shape = (mobilenet_input_size[0], mobilenet_input_size[1], 3)
num_classes = len(label_map)
model = build_mobilenetv2(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Predict on training and test data
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Convert predictions to class labels
train_predictions = np.argmax(train_predictions, axis=1)
test_predictions = np.argmax(test_predictions, axis=1)

# Compute confusion matrices
cm_train = confusion_matrix(y_train, train_predictions)
cm_test = confusion_matrix(y_test, test_predictions)

# Plot confusion matrices
def plot_confusion_matrices(cm_train, cm_test, class_names, filename='confusion_matrices.png'):
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman"],
    })
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.heatmap(cm_train, annot=True, fmt='d', ax=axes[0], cmap='Blues', xticklabels=class_names, yticklabels=class_names, cbar=False, annot_kws={"size": 16})
    axes[0].set_title('Training Confusion Matrix', fontsize=16)
    axes[0].set_xlabel('Predicted labels', fontsize=14)
    axes[0].set_ylabel('True labels', fontsize=14)
    sns.heatmap(cm_test, annot=True, fmt='d', ax=axes[1], cmap='Blues', xticklabels=class_names, yticklabels=class_names, cbar=False, annot_kws={"size": 16})
    axes[1].set_title('Testing Confusion Matrix', fontsize=16)
    axes[1].set_xlabel('Predicted labels', fontsize=14)
    axes[1].set_ylabel('True labels', fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close()

plot_confusion_matrices(cm_train, cm_test, list(label_map.keys()), filename='confusion_matrices.png')

# Plot convergence curves
def plot_convergence_curves(history, filename='convergence_curves.png'):
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman"],
    })
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Accuracy Convergence', fontsize=16)
    ax1.set_xlabel('Epoch', fontsize=14)
    ax1.set_ylabel('Accuracy', fontsize=14)
    ax1.legend()
    ax1.grid()
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Loss Convergence', fontsize=16)
    ax2.set_xlabel('Epoch', fontsize=14)
    ax2.set_ylabel('Loss', fontsize=14)
    ax2.legend()
    ax2.grid()
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close()

plot_convergence_curves(history, filename='convergence_curves.png')

# Calculate metrics for the training set
train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions, average='weighted')
train_recall = recall_score(y_train, train_predictions, average='weighted')
train_f1 = f1_score(y_train, train_predictions, average='weighted')

# Calculate metrics for the testing set
test_accuracy = accuracy_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions, average='weighted')
test_recall = recall_score(y_test, test_predictions, average='weighted')
test_f1 = f1_score(y_test, test_predictions, average='weighted')

# Create a DataFrame to display metrics as a table
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Training': [train_accuracy, train_precision, train_recall, train_f1],
    'Testing': [test_accuracy, test_precision, test_recall, test_f1]
})

print(metrics_df)
